In [1]:
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/'

import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support
import wandb
import os
import evaluate
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModelForSeq2SeqLM
from sklearn.dummy import DummyClassifier
from collections import Counter
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PrefixTuningConfig, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
from peft import PeftModel, PeftConfig
from glob import glob 
from collections import defaultdict, Counter
from os.path import basename


import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # replace the 0 with other gpu ids
os.environ["CUDA_VISIBLE_DEVICES"]="1"
torch.cuda.set_device(1)
device = "cuda:1"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/anaconda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda115.so
CUDA SETUP: CUDA runtime path found: /usr/lib/x86_64-linux-gnu/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 115
CUDA SETUP: Loading binary /opt/anaconda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda115.so...


# Label Politeness data with Appropriateness

In [2]:
se_df = pd.read_csv('/shared/2/projects/contextual-appropriateness/data/sepol.csv')
se_df['inst_id'] = range(0, 0+len(se_df))
se_df = se_df[se_df['label'].isin([1, -1])]
print(len(se_df))
se_df.head()

3302


,text,label,inst_id
4,Is `A` a global variable? What is x?,1,4
5,This is a very confusing question! How are yo...,-1,5
6,Why not using `isnan()` from math.h? Any speci...,-1,6
9,Does your project involve some graphical user ...,1,9
11,Usually compilers should generate a good code ...,1,11


In [3]:
wiki_df = pd.read_csv('/shared/2/projects/contextual-appropriateness/data/wikipol.csv')
wiki_df.drop_duplicates(inplace=True,ignore_index=True, subset='text', keep="first")
wiki_df = wiki_df[['label','text']]
wiki_df = wiki_df[wiki_df['label'].isin([1, -1])]
wiki_df['inst_id'] = range(0, 0+len(wiki_df))
print(len(wiki_df))
wiki_df.head()

2169


,label,text,inst_id
0,-1,Where did you learn English? How come you're t...,0
1,1,Thanks very much for your edit to the <url> ar...,1
5,1,"|style=""vertical-align: middle; padding: 3px;""...",2
8,-1,"These are my numbers: 7 years in Wikipedia, 6 ...",3
9,-1,I couldn't tell you why glam rock was there. B...,4


In [4]:
relationship_to_verbalization = {
    "friend": "a person to their friend",
    "parent":"a parent to their child",
    "child":"a child to their parent",
    "sibling":"a person to their sibling",
    "best friend":"a person to their best friend",
    "best_friend":"a person to their best friend",
    "neighbor":"a person to their neighbor",
    "coworker":"a person to their coworker",
    "boss":"a boss to their employee",
    "direct report (employee)":"a person to their boss",
    "direct_report":"a person to their boss",    
    "student":"a student to their teacher",
    "teacher":"a teacher to their student",
    "cousins":"a person to their cousin",
    "granparent":"a person to their grandchild",
    "grandparent":"a person to their grandchild",
    "grandchild":"a person to their grandparent",    
    "uncle/aunt":"an uncle/aunt to their niece or nephew",
    "uncle_aunt":"an uncle or aunt to their niece or nephew",    
    "neice_nephew":"a niece or nephew to their uncle or aunt",        
    "employee_in_large_company":"an employee in large company to another",
    "married":"a person to their spouse",
    "dating":"a person to the person they are dating",
    "engaged":"a person to their fiancee",
    "friends_with_benefits":"a person to their friend with benefits",
    "divorcee":"a person to their ex-spouse",
    "ex-girlfriend/ex-boyfriend":"a person to their ex-girlfriend or ex-boyfriend",
    "ex_dating":"a person to their ex-girlfriend or ex-boyfriend",    
    "step-sibling":"a person to their step-sibling",
    "step_sibling":"a person to their step-sibling",    
    "fan":"a fan to their hero",
    "hero":"a hero to their fan",
    "enemy":"a person to their enemy",
    "rival":"a person to their rival",
    "competitor":"a person to their competitor",
    "complete_stranger":"a person to a complete stranger",
    "acquaintance":"a person to an acquaintance",
    "person_with_authority":"a person with authority to a subordinate",
    "law_enforcement":"a member of law enforcement to a community member",
    "classmate":"a person to their classmate",
    "sports_teammate":"a person to their sports teammate",
    "club_member":"a person to a member of their club",
    "adopted_child":"an adopted child to their parent",
    "adopted child":"an adopted child to their parent",
    "domestic_partner":"a person to their domestic partner",
    "person_having_an_affair":"a person having an affair to their partner",
    "mentor":"a mentor to their mentee",
    "mentee":"a mentee to their mentor",
    "landlord":"a landlord to their tenant",
    "colleague":"a person to their colleague",
    "childhood_friend":"a person to their childhood friend",
    "old_friend":"a person to an old friend",
    "client":"a client to their lawyer",
    "lawyer":"a lawyer to their client",
    "patient":"a patient to their doctor",
    "doctor":"a doctor to their patient",
}
len(relationship_to_verbalization)

56

In [5]:
datadir = '/shared/2/projects/contextual-appropriateness/data/final-annotated-data-cleaned/'
train_df = pd.read_csv(datadir + 'train.csv')
rels = set(train_df.relationship)
len(rels)

49

In [6]:
def verbalize(rel, quote):
    if rel in relationship_to_verbalization:
        desc = relationship_to_verbalization[rel]
    else:
        desc = relationship_to_verbalization[rel.replace(' ', '_')]
    text = 'Rate whether it is inappropriate for this message to be said in the following social setting?\nsetting: from %s\nmessage: %s\nanswer (yes or no):' % (desc, quote)
    return text

In [19]:
all_rel_wiki_df = defaultdict(list)
for row in tqdm(wiki_df.itertuples(), total=len(wiki_df)):
    text = row.text
    label = row.label
    for rel in rels:
        all_rel_wiki_df['inst_id'].append(row.inst_id)
        all_rel_wiki_df['text'].append(verbalize(rel, text))
        all_rel_wiki_df['relationship'].append(rel)
        all_rel_wiki_df['label'].append(label)
        all_rel_wiki_df['answer'].append('yes' if label < 0 else 'no')
all_rel_wiki_df = pd.DataFrame(all_rel_wiki_df)


  0%|          | 0/2169 [00:00<?, ?it/s]

In [20]:
all_rel_se_df = defaultdict(list)
for row in tqdm(se_df.itertuples(), total=len(se_df)):
    text = row.text
    label = row.label
    for rel in rels:
        all_rel_se_df['inst_id'].append(row.inst_id)
        all_rel_se_df['text'].append(verbalize(rel, text))
        all_rel_se_df['relationship'].append(rel)
        all_rel_se_df['label'].append(label)
        all_rel_se_df['answer'].append('yes' if label < 0 else 'no')
all_rel_se_df = pd.DataFrame(all_rel_se_df)


  0%|          | 0/6603 [00:00<?, ?it/s]

In [25]:
len(all_rel_wiki_df), len(all_rel_se_df)

(106281, 323547)

In [22]:
all_rel_wiki_ds = Dataset.from_pandas(all_rel_wiki_df)
all_rel_se_ds = Dataset.from_pandas(all_rel_se_df)

politeness_dd = DatasetDict()
politeness_dd['wiki'] = all_rel_wiki_ds
politeness_dd['se'] = all_rel_se_ds

In [32]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

model_name_or_path = "google/flan-t5-xl"
tokenizer_name_or_path = "google/flan-t5-xl"
best_model_dir = '/shared/2/projects/contextual-appropriateness/models/peft/google/flan-t5-xl/seed-12345//best'

text_column = "text"
label_column = "answer"
max_length = 192
lr = 1e-2
num_epochs = 20
batch_size = 128

In [33]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=2, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

In [34]:
processed_politeness_dd = politeness_dd.map(
    preprocess_function,
    batched=True,
    num_proc=8,
    remove_columns=politeness_dd["wiki"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset (num_proc=8):   0%|          | 0/106281 [00:00<?, ? examples/s]

Running tokenizer on dataset (num_proc=8):   0%|          | 0/323547 [00:00<?, ? examples/s]

In [35]:
config = PeftConfig.from_pretrained(best_model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, best_model_dir)
model.to(device)
model.eval()
print('model loaded')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model loaded


In [36]:
wiki_dataloader = DataLoader(
    processed_politeness_dd['wiki'], shuffle=False, collate_fn=default_data_collator, 
    batch_size=154, pin_memory=True
)

wiki_preds = []
for step, batch in enumerate(tqdm(wiki_dataloader)):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    wiki_preds.extend(
        tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), 
                               skip_special_tokens=True)
    )  

  0%|          | 0/691 [00:00<?, ?it/s]

In [37]:
se_dataloader = DataLoader(
    processed_politeness_dd['se'], shuffle=False, collate_fn=default_data_collator, 
    batch_size=154, pin_memory=True
)

se_preds = []
for step, batch in enumerate(tqdm(se_dataloader)):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    se_preds.extend(
        tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), 
                               skip_special_tokens=True)
    )  

  0%|          | 0/2101 [00:00<?, ?it/s]

In [38]:
len(wiki_preds), len(all_rel_wiki_df)

(106281, 106281)

In [41]:
wiki_preds[:10]

['yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']

In [40]:
all_rel_wiki_df['predicted'] = wiki_preds

In [39]:
len(se_preds), len(all_rel_se_df)

(323547, 323547)

In [42]:
all_rel_se_df['predicted'] = se_preds

In [43]:
peft_outdir = '/shared/2/projects/contextual-appropriateness/results/peft/'
all_rel_wiki_df.to_csv(peft_outdir + 'politeness.wiki.labeled.csv', index=False)
all_rel_se_df.to_csv(peft_outdir + 'politeness.se.labeled.csv', index=False)